## **Calculate MitoNet performance metrics on FAST-EM data**
---
Calculate semantic and instance performance metrics of MitoNet on FAST-EM data

|   | Dataset                   | Figure | Dwell time (µs) | No. of sections | ROA size (µm) |
|---|---------------------------|--------|-----------------|-----------------|---------------|
|   | 20230711_MCF7_UAc         | 6      | 20              | 54              | 240 x 240     |
|   | 20230626_Rat_Pancreas_OTO | S5     | 10              | 44              | 96 x 96       |

In [1]:
# Data import
import webknossos as wk
import numpy as np

# Other routines
import src.importo
import src.performance_metrics

### **3.1 Configure dataset parameters**
---

#### **Dataset name**
---
Uncomment the one to analyze

In [2]:
# dataset_full_url = 'https://webknossos.tnw.tudelft.nl/links/5Q_PJXjPPa4P3XBO' # 20230626_Rat_Pancreas_OTO
dataset_full_url = 'https://webknossos.tnw.tudelft.nl/links/_eSz0uGGSzkZ1w__' # 20230711_MCF7_UAc

## Mag 
mag_x_seg, mag_y_seg, mag_z_seg = 4, 4, 1 # For MitoNet predictions 4, 4, 1 or larger, no 2, 2, 1 and 1, 1, 1 exist!
mag_x_EM, mag_y_EM, mag_z_EM = 4, 4, 1 # Show data at 4, 4, 1 Mag (4x downsampled / 16nm/pixel)

MAG_seg = wk.Mag(f"{mag_x_seg}-{mag_y_seg}-{mag_z_seg}")
MAG_EM = wk.Mag(f"{mag_x_EM}-{mag_y_EM}-{mag_z_EM}")

#### **Layers and bounding boxes**
---
- Uncomment under the dataset header the layers to analyze. Either select "fine aligned data" or "SOFIMA aligned data" block 
- Bounding boxes apply to specific views of the data with ground truth annotations. `20230626_Rat_Pancreas_OTO`: use either bounding box 1 or 2, for `20230711_MCF7_UAc` there is only one bounding box

`20230626_Rat_Pancreas_OTO`

In [3]:
## FINE ALIGNED DATA
# EM_layer = 'EM'
# GT_layer = 'Mito_GT_EM'
# mitonet_layer = 'EM_MitoNet'

## SOFIMA ALIGNED DATA
# EM_layer = 'EM_realigned_SOFIMA'
# GT_layer = 'Mito_GT_EM_realigned_SOFIMA'
# mitonet_layer = 'EM_realigned_SOFIMA_MitoNet'

In [ ]:
# ## BBOX 1 (RAT PANCREAS)
# bbox_EM = wk.BoundingBox((14868, 13908, 0),
#                           (2141, 2210, 38)).align_with_mag(MAG_seg) # ((x0, y0, z0), (x_size, y_size, z_size))
# bbox_seg = wk.BoundingBox((14868, 13908, 0),
#                           (2141, 2210, 38)).align_with_mag(MAG_seg) # ((x0, y0, z0), (x_size, y_size, z_size))

## BBOX 2 (RAT PANCREAS)
# bbox_EM = wk.BoundingBox((17277, 21620, 0),
#                           (2141, 1963, 43)).align_with_mag(MAG_seg) # ((x0, y0, z0), (x_size, y_size, z_size))
# bbox_seg = wk.BoundingBox((17277, 21620, 0),
#                           (2141, 1963, 43)).align_with_mag(MAG_seg) # ((x0, y0, z0), (x_size, y_size, z_size))

`20230711_MCF7_UAc`

In [6]:
## FINE ALIGNED DATA
EM_layer = 'EM'
GT_layer = 'Mito_GT_EM'
mitonet_layer = 'GT_EM_MitoNet'

## SOFIMA ALIGNED DATA
# EM_layer = 'GT_EM_realigned_SOFIMA'
# GT_layer = 'Mito_GT_EM_realigned_SOFIMA'
# mitonet_layer = 'GT_EM_realigned_SOFIMA_MitoNet'

In [7]:
# BBOX 1 (MCF-7 cells)
bbox_EM = wk.BoundingBox((25184, 52192, 0),
                         (4416, 8256, 54)).align_with_mag(MAG_EM) # ((x0, y0, z0), (x_size, y_size, z_size))
bbox_seg = wk.BoundingBox((25184, 52192, 0),
                          (4416, 8256, 54)).align_with_mag(MAG_seg) # ((x0, y0, z0), (x_size, y_size, z_size))

### **3.2 Read data from WebKnossos (remote)**
---
Reads data, segmentations, and ground truth annotations from remote `WebKnossos` dataset

In [4]:
# Read dataset
dataset, voxelsize = src.importo.import_wk_dataset_remote(dataset_full_url=dataset_full_url)
scale_factor = voxelsize[2] / voxelsize[0] / (mag_x_EM / mag_z_EM) # determine scalefactor for viewing

In [8]:
EM = dataset.get_layer(EM_layer) # EM Layer
seg = dataset.get_layer(mitonet_layer) # MitoNet layer
gt = dataset.get_layer(GT_layer) # GT layer

mag_view = EM.get_mag(MAG_EM) # MagView
mag_view_seg = seg.get_mag(MAG_seg) # MagView
mag_view_gt = gt.get_mag(MAG_seg) # MagView

view = mag_view.get_view(absolute_offset=bbox_EM.topleft, size=bbox_EM.size) # "absolute_offset" and "size" are in Mag(1)!
view_seg = mag_view_seg.get_view(absolute_offset=bbox_seg.topleft, size=bbox_seg.size) # "absolute_offset" and "size" are in Mag(1)!
view_gt = mag_view_gt.get_view(absolute_offset=bbox_seg.topleft, size=bbox_seg.size) # "absolute_offset" and "size" are in Mag(1)!
    
data = view.read()
seg = view_seg.read()
gt = view_gt.read()

In [9]:
data, seg, gt = np.squeeze(data), np.squeeze(seg), np.squeeze(gt)
print(data.shape) # Coordinates are (x, y, z), but Napari weirdly reads this as (y, z, x)...
print(seg.shape) # Coordinates are (x, y, z), but Napari weirdly reads this as (z, x, y)...
print(gt.shape)# Coordinates are (x, y, z), but Napari weirdly reads this as (z, x, y)...

(1104, 2064, 54)
(1104, 2064, 54)
(1104, 2064, 54)


In [10]:
# Pick segmentations where GT exists for evaluation
seg_mitonet_filtered = np.where(gt > 0, seg, 0)

# Include also predicted pixels of objects outside of GT pixels
object_ids = np.unique(seg_mitonet_filtered)
objects = np.isin(seg, list(object_ids))
seg_mitonet_only_GT_objects = np.where(objects, seg, 0)

### 3.2 **Compute performance metrics**
---
Computes (semantic) IoU. Then computes IoU matrix for all objects by iterating over all unique labels (Mitochondria instances) in both the GT and the MitoNet predictions and separating their masks. The IoU matrix is a one-to-one computation of all mask pairs and their IoU. Then, the masks are matched based on the highest IoU using the Hungarian algorithm as implemented in `scipy.optimize.linear_sum_assignment`. WARNING: Problem complexity scales poorly with volume size, this takes a long time to compute for the cell dataset (several hours)!!!

Provided in the folder `iou_matrices` are the precomputed IOU matrics that can be used to evaluate the F1 and AP scores directly. 

With the IOU matrices, the following is computed
- IoU
- F1@50
- F1@75
- AP@50
- AP@75

In [11]:
# Reduce memory footprint
gt = gt.astype('uint16')
seg = seg.astype('uint16')

In [97]:
# IoU score
seg_semantic = np.where(seg_mitonet_only_GT_objects > 0, 1, 0)
gt_semantic = np.where(gt > 0, 1, 0)
iou = src.performance_metrics.compute_iou(seg_semantic, gt_semantic)

### COMPUTATIONALLY EXPENSIVE

In [14]:
# Separate instances and compute IoU matrix for matching instances. 
print('Separating prediction masks...')
pred_masks = src.performance_metrics.separate_masks(seg)
print('Separating GT masks...')
gt_masks = src.performance_metrics.separate_masks(gt)
print('Matching instances...')

# Compute IoU matrix (This step takes the most time)
iou_matrix = src.performance_metrics.compute_iou_matrix(pred_masks, gt_masks)

In [99]:
### Can save IoU matrix to avoid running the computation multiple times...
# np.save('iou_matrix_cells_rigid', iou_matrix)

In [12]:
### Load IoU matrix if not performing the computation
# iou_mat = np.load('iou_matrix_cells_realigned.npy')

In [107]:
# F1 score, AP score @ IoU thres
IoU_thres=0.50
tp, fp, fn, matches = src.performance_metrics.match_instances(pred_masks, gt_masks, iou_matrix, IoU_thres)

Applying Hungarian algorithm...
Finding matches...


In [108]:
f1_= src.performance_metrics.compute_f1(tp, fp, fn)
ap_= src.performance_metrics.compute_ap(tp, fp, fn)
scores = {
    "IoU": iou,
    f"F1@{int(100*IoU_thres)} score": f1_,
    f"AP@{int(100*IoU_thres)} score": ap_
}
print(scores)

{'IoU': 0.7010335736051205, 'F1@50 score': 0.41964285714285715, 'AP@50 score': 0.2655367231638418}


In [51]:
print(tp, fp, fn)

64 32 24
